In [125]:
import sys
import numpy as np

sys.path.append('../../lib/current/')

## Neural Network from Scratch - 2

we shall use `Neuron` from [Neuron.py](../../lib/release/v0.1.11-beta/src/neuron/Neuron.py).

and previous knowledge from [1_study.ipynb](1_study.ipynb)

In [126]:
from src.neuron.Neuron import Neuron
from src.core.Activation import Activation
from collections.abc import Iterable
from numbers import Number

In [127]:
class Layer:
  # constructor
  def __init__(self, count:int, input_size:int, activation:str|Activation):
    self.neurons = [
      Neuron(
        input_size=input_size,
        activation=activation
      ) for _ in range(count)
    ]

  # forward
  def forward(self, inputs):
    outputs = np.array([neuron.forward(inputs) for neuron in self.neurons])
    return outputs

  # backward
  def backward(self, errors, learning_rate):
    next_errors = np.zeros(len(self.neurons[0].weights))
    
    for i, neuron in enumerate(self.neurons):
      next_errors += neuron.backward(error=errors[i], learning_rate=learning_rate)
    
    return next_errors


In [128]:
class Sequential:
  def __init__(self, layers:Iterable=None):
    self.layers = layers or []

  def add(self, layer:Layer):
    self.layers.append(layer)

  def forward(self, inputs:Iterable):
    for layer in self.layers:
      inputs = layer.forward(inputs)
    return inputs

  def backward(self, error:Number|Iterable, learning_rate:Number):
    for layer in reversed(self.layers):
      error = layer.backward(error, learning_rate)

  def train(self, X, y, loss, epochs, learning_rate):
    for _ in range(epochs):
      for i in range(len(X)):
        output = self.forward(X[i])
        error = loss.derivative(y[i], output)
        self.backward(error, learning_rate)


## Usage

In [129]:
from src.activation.Sigmoid      import Sigmoid
from src.losses.MeanSquaredError import MeanSquaredError

In [130]:
model = Sequential([
  Layer(3, 3, Sigmoid()),
  Layer(1, 3, Sigmoid()),
])

In [131]:
from dataset import X, Y

In [132]:
model.train(
  X, Y,
  loss=MeanSquaredError(),
  epochs=2000,
  learning_rate=0.1
)

In [133]:
# testing
for i in range(len(X)):
  print('predicted =', round(model.forward(X[i])[0], 3), 'actual =', Y[i])

predicted = 0.99 actual = 1
predicted = 0.005 actual = 0
predicted = 1.0 actual = 1
predicted = 0.005 actual = 0
predicted = 0.006 actual = 0
predicted = 0.005 actual = 0
predicted = 0.99 actual = 1
predicted = 1.0 actual = 1
predicted = 1.0 actual = 1
predicted = 0.005 actual = 0
